In [1]:
import os
import math
import random
import numpy as np
import pandas as pd

from sklearn import preprocessing
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Model, Sequential, initializers
from tensorflow.keras.layers import LSTM, Dense

import matplotlib.pyplot as plt
import matplotlib as mpl

In [2]:
seed_value = 1
os.environ['PYTHONHASHSEED']=str(seed_value)
random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)

In [3]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    """
    Frame a time series as a supervised learning dataset.
    Arguments:
        data: Sequence of observations as a list or NumPy array.
        n_in: Number of lag observations as input (X).
        n_out: Number of observations as output (y).
        dropnan: Boolean whether or not to drop rows with NaN values.
    Returns:
        Pandas DataFrame of series framed for supervised learning.
    """
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [4]:
def graph_data(df, groups):
    plt.figure(figsize=(16,9))
    i = 1
    for group in groups:
        plt.subplot(len(groups), 1, i)
        plt.plot(df['Day'], df[group])
        plt.title(group, y=0.5, loc='left')
        i += 1
    plt.show()

In [5]:
def scale(X_total):
    scaler = preprocessing.MinMaxScaler(feature_range=(0, 1)).fit(X_total)
    X_total_scaled = scaler.transform(X_total)
    return scaler, X_total_scaled

In [6]:
def invert_scale(scaler, X, yhat):
    new_row = [x for x in X] + [yhat]
    array = numpy.array(new_row)
    array = array.reshape(1, len(array))
    inverted = scaler.inverse_transform(array)
    return inverted[0, -1]

In [7]:
def fit_lstm(train, batch_size, nb_epoch, neurons, timesteps):
    X, y = train[:, 0:-1], train[:, -1]
    X = X.reshape(X.shape[0], timesteps, 1)
    model = keras.Sequential()
    model.add(keras.layers.LSTM(neurons, batch_input_shape=(batch_size, X.shape[1], X.shape[2])))
    model.add(keras.layers.Dense(12))
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mse'])
    for i in range(nb_epoch):
        model.fit(X, y, epochs=1, batch_size=batch_size, verbose=0, shuffle=False)

In [8]:
saltlake_week = pd.read_csv('Data/saltlake_week (3).csv')

saltlake_week.head()

,dates,Cases,VMT (Veh-Miles),VMT Truck,VMT Total,News Sentiment,Avg News Sentiment,Unemployment Rate,PRCP,SNOW,...,Percent_Fully_Vaccinated_5&Older,Percent_Fully_Vaccinated_12&Older,Stay at Home,Mask,School Opening,Health Emergency,TAVG,TMAX,TMIN,Day
0,201902,0,66181817.1,3867774.69,65992456.97,-0.591161,-0.084452,0.029341,0.07,0.4,...,NaN,NaN,0.0,0.0,1.0,0.0,32.285714,36.571429,28.571429,1/7/2019
1,201903,0,65217107.0,3800585.14,65027746.62,-0.718490,-0.102641,0.029341,0.85,1.0,...,NaN,NaN,0.0,0.0,1.0,0.0,33.428571,39.857143,27.857143,1/14/2019
2,201904,0,63764156.8,3764387.42,63574796.54,-0.797337,-0.113905,0.029341,0.62,4.2,...,NaN,NaN,0.0,0.0,1.0,0.0,31.571429,40.285714,24.571429,1/21/2019
3,201905,0,68255092.9,3931200.14,68065732.66,-0.792740,-0.113249,0.027660,0.07,0.0,...,NaN,NaN,0.0,0.0,1.0,0.0,36.142857,46.285714,29.714286,1/28/2019
4,201906,0,60363403.4,3582669.45,60174043.18,-0.328870,-0.046981,0.027660,0.60,9.8,...,NaN,NaN,0.0,0.0,1.0,0.0,31.285714,37.000000,22.285714,2/4/2019


In [9]:
saltlake_week.isnull().sum()

dates                                  0
Cases                                  0
VMT (Veh-Miles)                        0
VMT Truck                             22
VMT Total                             22
News Sentiment                         0
Avg News Sentiment                     0
Unemployment Rate                      0
PRCP                                   0
SNOW                                   0
SNWD                                   0
Percent_Fully_Vaccinated_Total       101
Percent_Fully_Vaccinated_5&Older     101
Percent_Fully_Vaccinated_12&Older    101
Stay at Home                           0
Mask                                  22
School Opening                        22
Health Emergency                      22
TAVG                                   0
TMAX                                   0
TMIN                                   0
Day                                    0
dtype: int64

In [10]:
saltlake_week.fillna(0, inplace=True)
saltlake_week.head()

,dates,Cases,VMT (Veh-Miles),VMT Truck,VMT Total,News Sentiment,Avg News Sentiment,Unemployment Rate,PRCP,SNOW,...,Percent_Fully_Vaccinated_5&Older,Percent_Fully_Vaccinated_12&Older,Stay at Home,Mask,School Opening,Health Emergency,TAVG,TMAX,TMIN,Day
0,201902,0,66181817.1,3867774.69,65992456.97,-0.591161,-0.084452,0.029341,0.07,0.4,...,0.0,0.0,0.0,0.0,1.0,0.0,32.285714,36.571429,28.571429,1/7/2019
1,201903,0,65217107.0,3800585.14,65027746.62,-0.718490,-0.102641,0.029341,0.85,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,33.428571,39.857143,27.857143,1/14/2019
2,201904,0,63764156.8,3764387.42,63574796.54,-0.797337,-0.113905,0.029341,0.62,4.2,...,0.0,0.0,0.0,0.0,1.0,0.0,31.571429,40.285714,24.571429,1/21/2019
3,201905,0,68255092.9,3931200.14,68065732.66,-0.792740,-0.113249,0.027660,0.07,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,36.142857,46.285714,29.714286,1/28/2019
4,201906,0,60363403.4,3582669.45,60174043.18,-0.328870,-0.046981,0.027660,0.60,9.8,...,0.0,0.0,0.0,0.0,1.0,0.0,31.285714,37.000000,22.285714,2/4/2019


In [ ]:
graph_data(saltlake_week, ['Cases', 'VMT (Veh-Miles)', 'News Sentiment', 'Unemployment Rate',
           'Percent_Fully_Vaccinated_5&Older'])

In [11]:
X_total = saltlake_week[['Cases', 'VMT (Veh-Miles)', 'News Sentiment', 'Unemployment Rate', 'PRCP', 'SNOW', 'SNWD',
                         'Percent_Fully_Vaccinated_5&Older', 'TAVG', 'Stay at Home', 'Mask', 'School Opening', 'Health Emergency']]
X_total.shape

(152, 13)

In [12]:
X_total.isnull().sum()

Cases                               0
VMT (Veh-Miles)                     0
News Sentiment                      0
Unemployment Rate                   0
PRCP                                0
SNOW                                0
SNWD                                0
Percent_Fully_Vaccinated_5&Older    0
TAVG                                0
Stay at Home                        0
Mask                                0
School Opening                      0
Health Emergency                    0
dtype: int64

In [13]:
scaler, data = scale(X_total)
data

array([[0.        , 0.39618322, 0.68322341, ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.36733388, 0.66073054, ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.32388387, 0.64680217, ..., 0.        , 1.        ,
        0.        ],
       ...,
       [0.43184475, 0.81289536, 0.8292505 , ..., 0.        , 0.        ,
        0.        ],
       [0.31587561, 0.57519729, 0.86246876, ..., 0.        , 0.        ,
        0.        ],
       [0.43558569, 0.79595918, 0.83061445, ..., 0.        , 0.        ,
        0.        ]])

In [14]:
n_weeks=1
n_features=13
n_obs = n_weeks*n_features

In [15]:
data = series_to_supervised(data, n_in=n_weeks, n_out=1, dropnan=True)
data

,var1(t-1),var2(t-1),var3(t-1),var4(t-1),var5(t-1),var6(t-1),var7(t-1),var8(t-1),var9(t-1),var10(t-1),...,var4(t),var5(t),var6(t),var7(t),var8(t),var9(t),var10(t),var11(t),var12(t),var13(t)
1,0.000000,0.396183,0.683223,0.121382,0.039773,0.030075,0.236162,0.000000,0.087470,0.0,...,0.121382,0.482955,0.075188,0.044280,0.000000,0.106383,0.0,0.0,1.0,0.0
2,0.000000,0.367334,0.660731,0.121382,0.482955,0.075188,0.044280,0.000000,0.106383,0.0,...,0.121382,0.352273,0.315789,0.490775,0.000000,0.075650,0.0,0.0,1.0,0.0
3,0.000000,0.323884,0.646802,0.121382,0.352273,0.315789,0.490775,0.000000,0.075650,0.0,...,0.103062,0.039773,0.000000,0.000000,0.000000,0.151300,0.0,0.0,1.0,0.0
4,0.000000,0.458184,0.647614,0.103062,0.039773,0.000000,0.000000,0.000000,0.151300,0.0,...,0.103062,0.340909,0.736842,0.704797,0.000000,0.070922,0.0,0.0,1.0,0.0
5,0.000000,0.222185,0.729557,0.103062,0.340909,0.736842,0.704797,0.000000,0.070922,0.0,...,0.103062,0.261364,0.248120,0.450185,0.000000,0.122931,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147,0.411503,0.817572,0.731565,0.000000,0.704545,0.000000,0.000000,0.956102,0.404255,0.0,...,0.065972,0.034091,0.000000,0.000000,0.963498,0.406619,0.0,0.0,0.0,0.0
148,0.430676,0.793182,0.785753,0.065972,0.034091,0.000000,0.000000,0.963498,0.406619,0.0,...,0.065972,0.142045,0.000000,0.000000,0.969927,0.330969,0.0,0.0,0.0,0.0
149,0.446925,0.795731,0.845507,0.065972,0.142045,0.000000,0.000000,0.969927,0.330969,0.0,...,0.065972,0.017045,0.000000,0.000000,0.975820,0.257683,0.0,0.0,0.0,0.0
150,0.431845,0.812895,0.829250,0.065972,0.017045,0.000000,0.000000,0.975820,0.257683,0.0,...,0.065972,0.000000,0.000000,0.000000,0.981235,0.151300,0.0,0.0,0.0,0.0


In [16]:
values = data.values
values.shape

(151, 26)

In [17]:
X = values[:, :n_obs]
y = values[:, -n_features:]

print(X.shape, len(X), y.shape)

(151, 13) 151 (151, 13)


In [18]:
m, n = X.shape
m, n

(151, 13)

In [19]:
train = values[:147, :]
test = values[147:, :]

print(train.shape)
print(test.shape)

(147, 26)
(4, 26)


In [20]:
X_train = train[:, :n_obs]
y_train = train[:, -n_features:]

print(X_train.shape)
print(y_train.shape)

(147, 13)
(147, 13)


In [21]:
X_test = test[:, :n_obs]
y_test = test[:, -n_features:]

print(X_test.shape)
print(y_test.shape)

(4, 13)
(4, 13)


In [22]:
X_train = X_train.reshape((X_train.shape[0], n_weeks, n_features))
X_test = X_test.reshape((X_test.shape[0], n_weeks, n_features))
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(147, 1, 13) (147, 13) (4, 1, 13) (4, 13)


In [23]:
X_total.shape

(152, 13)

In [24]:
rmse_benchmark = math.sqrt(mean_squared_error(X_total.iloc[148:, 1]/1000000, X_total.iloc[146:150, 1]/1000000))
mape_benchmark = math.sqrt(mean_absolute_percentage_error(X_total.iloc[148:, 1]/1000000, X_total.iloc[146:150, 1]/1000000))

rmse_ref = rmse_benchmark
mape_ref = mape_benchmark

print(rmse_ref)
print(mape_ref)

3.730716954939215
0.17798542752702134


In [25]:
OUTPUT_PATH = 'output'
EPOCHS = 200
BS = 32
EARLY_STOPPING_PATIENCE = 10

In [ ]:
mpl.use('Agg')

def save_plot(H, path):
    # plot the training loss and accuracy
    plt.style.use("ggplot")
    plt.figure()
    plt.plot(H.history["loss"], label="train_loss")
    plt.plot(H.history["val_loss"], label="val_loss")
    plt.title("Training Loss and Accuracy")
    plt.xlabel("Epoch #")
    plt.ylabel("Loss/Accuracy")
    plt.legend()
    plt.savefig(path)

In [ ]:
import keras_tuner as kt
from keras_tuner import BayesianOptimization
from tensorflow.keras.callbacks import EarlyStopping

def build_model(hp):
    model = keras.Sequential()
    model.add(keras.layers.LSTM(units=hp.Int('lstm_1',min_value=10,
                                        max_value=100,
                                        step=10),
                                input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(keras.layers.Dropout(0.2))
    model.add(keras.layers.Dense(13))
    
    lr = hp.Choice('learning_rate', 
                   values=[1e-1, 3e-1, 1e-2, 3e-2, 1e-3, 3e-3, 1e-4, 3e-4, 1e-5])
    
    model.compile(loss='mse', optimizer=keras.optimizers.Adam(learning_rate = lr),
                   metrics=['mse'])
    return model

In [ ]:
es = EarlyStopping(
    monitor="val_loss",
    patience=EARLY_STOPPING_PATIENCE,
    restore_best_weights=True)

In [ ]:
print("[INFO] instantiating a bayesian optimization tuner object...")
tuner = kt.BayesianOptimization(
    build_model,
    objective="mse",
    max_trials=10,
    seed=42,
    directory=OUTPUT_PATH,
    project_name="bayesian")

In [ ]:
print("[INFO] performing hyperparameter search...")
tuner.search(
    x=X_train, y=y_train,
    # validation_data=(X_test, y_test),
    validation_split=0.2,
    batch_size=BS,
    callbacks=[es],
    epochs=EPOCHS
)

bestHP = tuner.get_best_hyperparameters(num_trials=1)[0]
print("[INFO] optimal number of nodes in lstm_1 layer: {}".format(
    bestHP.get("lstm_1")))
print("[INFO] optimal learning rate: {}".format(
    bestHP.get("learning_rate")))

In [ ]:
print("[INFO] training the best model...")
model = tuner.hypermodel.build(bestHP)
H = model.fit(
    x=X_train, y=y_train, 
    validation_data=(X_test, y_test),
    batch_size=BS,
    callbacks=[es],
    epochs=EPOCHS
)

print("[INFO] evaluating network...")
predictions = model.predict(x=X_test, batch_size=BS)
print(model.evaluate(X_test, y_test))

save_plot(H, "output/bayesian")